<a href="https://colab.research.google.com/github/susan8653/TCGAdata_LUAD-LUSC_analysis/blob/master/import_install_packages_Datadownload_labeling_Z_score_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install package

In [ ]:
!pip install selenium

!apt-get update

!apt install chromium-chromedriver

In [ ]:
!pip install catboost

#import package

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import tarfile
import math
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import time
import sys
import shutil

from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

#Data download_Crawling

In [ ]:
def download(x):
  for i in x:
    url = 'http://gdac.broadinstitute.org/runs/stddata__2016_01_28/data/{0}/20160128/gdac.broadinstitute.org_{0}.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.Level_3.2016012800.0.0.tar.gz'.format(i)
    driver.get(url)
download(genome_name)

import shutil
for i in genome_name:
  shutil.move('/content/gdac.broadinstitute.org_{0}.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.Level_3.2016012800.0.0.tar.gz'.format(i),path)

In [ ]:
def Load_Dataframe(path, genome_list):
  tar_list = os.listdir(path) #다운로드 된 경로 
  genome_path = f'{path}../genome_data/' # 같은 위치에 파일 생성
  df = pd.DataFrame()

  for idx,f in enumerate(tar_list):
    genome = genome_list[idx] # 암 명칭 입력 
    # 다운받은 파일 압축해제
    ap = tarfile.open(path + f) 
    ap.extractall(genome_path) # 압축 해제 된 파일 저장
    ap.close 
    # 폴더 명 암 이름으로 변경
    f_old = os.path.join(genome_path, f.split('.tar')[0]) 
    f_new = os.path.join(genome_path, f'{genome}')
    os.rename(f_old, f_new) 
    # 암 txt 파일 불러오기
    txt_list = os.listdir(f'{genome_path}{genome}/')
    file = ''.join([i for i in txt_list if str(genome) in i])
    data = pd.read_csv(f'{genome_path+genome}/{file}', sep='\t', low_memory=False, index_col='Hybridization REF', skiprows=[1])
    
  return data

#Data labeling & Loading

In [ ]:
def Load_Dataframe(path, genome_list):
  df = pd.DataFrame() # 암 데이터를 합쳐서 출력 받을 최종 df

  path_list= os.listdir(path) # 압축해제된 파일 경로
  for idx, f in enumerate(path_list):
    genome = genome_list[idx] # 불러올 암 이름 입력 받기
    genome_file = os.listdir(f'{path}/{genome}/') # 해당 암의 폴더로 들어가기
    txt = ''.join([i for i in genome_file if str(genome)in i]) # 필요한 파일만 출력
    # 파일 불러오기
    data = pd.read_csv(f'{path}/{genome}/{txt}', sep='\t', low_memory=False, index_col='Hybridization REF', skiprows=[1])
    data = data.transpose()
    #데이터 라벨링
    data['Target'] = data.index.str[13:15]
    data['Target'] = data['Target'].replace({'01':'Tumor', '02':'Tumor', '11':'Normal', '10':'Normal'})
    data['Target'] = genome +'_'+ data['Target']
    df = pd.concat([df, data])
    
  return df

#z-score

In [ ]:
def z(genome_list):
  d1 = pd.DataFrame()  #최종 data

  for idx, g in enumerate(genome_list):
    data_select = data_Tumor[data_Tumor['Target']==idx]
    Topdf = data_select[topDf.index]
    #Topdf
    
    #Z-Score
    z_score = (Topdf - Topdf.mean())/Topdf.std()
    z_score = z_score.set_axis([g]*len(z_score), axis=0)
    z_score = z_score.transpose()
    d1 = pd.concat([d1, z_score], axis=1)

  return d1